# Fallstudie

## Import Pakete

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sqlalchemy import create_engine 
from sklearn.neighbors import KNeighborsClassifier
import pickle

## Import Datenbank

In [2]:

cnx = create_engine('sqlite:///Kundendaten.db').connect() 

  
df = pd.read_sql_table('testdaten', cnx) 

In [3]:
#df = df.head(100000)

## Datenmodellierung 

In [4]:
df = df.drop(["Datum"], 1)
df = df.drop(["Anzahl"], 1)
df = df.drop(["Gewinn"], 1)
df = df.drop(["Jahr"], 1)
df = df.drop(["Monat"], 1)
df = df.drop(["Tag"], 1)
df = df.drop(["index"], 1)

In [5]:
df.head()

,Alter,Geschlecht,Job,Familienstand,Kinder,Gehalt,Angebotenes Produkt,Gekauft
0,52,M,Öffentlicher Dienst,ledig,ja,54000.0,Depotkonto,nein
1,57,M,Ingenieurswesen,verheiratet,ja,167000.0,Depotkonto,nein
2,24,W,Handel,ledig,nein,21000.0,Kredit,nein
3,32,M,Handwerk,verheiratet,ja,32000.0,Altersvorsorge,nein
4,62,W,Handel,verheiratet,ja,64000.0,Altersvorsorge,nein


In [6]:
pd.options.mode.chained_assignment = None  # default='warn'
df["Altersgruppe"] = ">65"

df["Altersgruppe"][df["Alter"] > 65] = ">65"
df["Altersgruppe"][(df["Alter"] >= 50) & (df["Alter"] < 65)] = "50 - 65"
df["Altersgruppe"][(df["Alter"] >= 30) & (df["Alter"] < 50)] = "30 - 49"
df["Altersgruppe"][(df["Alter"] >= 18) & (df["Alter"] < 30)] = "18 - 29"
df["Altersgruppe"][df["Alter"] < 18] = "<18"

In [7]:
df["balance_cat"] = "> 100000"

df["balance_cat"][df["Gehalt"] > 100000] = "> 100000"
df["balance_cat"][(df["Gehalt"] >= 80000) & (df["Gehalt"] < 100000)] = "80000 - 99999"
df["balance_cat"][(df["Gehalt"] >= 60000) & (df["Gehalt"] < 80000)] = "60000 - 79999"
df["balance_cat"][(df["Gehalt"] >= 40000) & (df["Gehalt"] < 60000)] = "40000 - 59999"
df["balance_cat"][(df["Gehalt"] >= 20000) & (df["Gehalt"] < 40000)] = "20000 - 39999"
df["balance_cat"][(df["Gehalt"] >= 0) & (df["Gehalt"] < 20000)] = "0 - 19999"
df["balance_cat"][df["Gehalt"] < 0] = "< 0"

In [8]:
df["Produkt"] = df["Angebotenes Produkt"]
df["Produkt"] = df["Produkt"].replace(["Girokonto", "Kredit","Tagesgeldkonto","Depotkonto", "Altersvorsorge","Versicherung", "Bausparvertrag"],[1,2,3,4,5,6,7])
df = df.drop(["Angebotenes Produkt"], 1)

In [9]:

df["Job"] = df["Job"].replace(["Studium", "Öffentlicher Dienst", "Rente", "Informatik", "Handel", "Handwerk", "Administrativ", "Ingenieurswesen", "Management", "Arbeitslos"],[0,1,2,3,4,5,6,7,8,9])

In [10]:
df["Geschlecht"] = df["Geschlecht"].replace(["M", "W", "D"],[1,2,3])

In [11]:
df["Familienstand"] = df["Familienstand"].replace(["verheiratet", "ledig", "aufgelöste Beziehung"],[1,2,3])

In [12]:
df["Kinder"] = df["Kinder"].replace(["ja", "nein"],[1,0])

In [13]:
df["Altersgruppe"] = df["Altersgruppe"].replace(["<18", "18 - 29", "30 - 49", "50 - 65", ">65"],[0,1,2,3,4])

In [14]:
df["balance_cat"] = df["balance_cat"].replace(["< 0", "0 - 19999", "20000 - 39999", "40000 - 59999", "60000 - 79999", "80000 - 99999", "> 100000"],[1,2,3,4,5,6,7])
df = df.drop(["Gehalt"], axis=1)

In [15]:
df = df.drop(["Alter"], axis=1)

In [16]:
df["Gekauft"] = df["Gekauft"].replace(["nein", "ja"],[0, 1])

In [17]:
labels = np.array(df["Gekauft"])
#labels = np.array(df["Produkt"])

In [18]:
dfTest = df

In [19]:
df = df.drop(["Gekauft"], axis=1)
#df = df.drop(["Produkt"], axis=1)

In [20]:
df_list = list(df.columns)

## Algorithmen

### Trainings- und Testdaten vorbereiten

In [21]:
df = np.array(df)

In [22]:
train_data, test_data, train_labels, test_labels = train_test_split(df, labels, test_size = 0.25, random_state = 42)

In [23]:
print('Training Features Shape:', train_data.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_data.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (58551, 7)
Training Labels Shape: (58551,)
Testing Features Shape: (19518, 7)
Testing Labels Shape: (19518,)


### RANDOM FOREST CLASSIFIER

In [24]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [25]:
rf.fit(train_data, train_labels);

In [26]:
pred_rf = rf.predict(test_data)


In [27]:
errors_rf = abs(pred_rf - test_labels)
errors_rf = [x for x in errors_rf if x != 0]

In [28]:
print(f"Accuracy RF = {(len(test_labels)-len(errors_rf))/len(test_labels)}")

Accuracy RF = 0.7293780100420125


In [29]:
confusion_matrix_rf = sklearn.metrics.confusion_matrix(test_labels, pred_rf)
tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix_rf.ravel()

Precision_rf = tp_rf/(tp_rf+fp_rf)
Recall_rf = tp_rf/(tp_rf+fn_rf)
F1_rf = 2*((Precision_rf*Recall_rf)/(Precision_rf+Recall_rf))

print(f"Precision RF = {Precision_rf}")
print(f"Recall Rf = {Recall_rf}")
print(f"F1 Score RF = {F1_rf}")

Precision RF = 0.5078909612625538
Recall Rf = 0.2658655651520841
F1 Score RF = 0.34902637416810445


### K NEAREST NEIGHBORS

In [30]:
classifier = KNeighborsClassifier(n_neighbors=7)

In [31]:
classifier.fit(train_data, train_labels)

KNeighborsClassifier(n_neighbors=7)

In [32]:
pred_knn = classifier.predict(test_data)

In [33]:
errors_knn = abs(pred_knn - test_labels)
errors_knn = [x for x in errors_knn if x != 0]

In [34]:
print(f"Accuracy KNN = {(len(test_labels)-len(errors_knn))/len(test_labels)}")

Accuracy KNN = 0.7164668511117942


In [35]:
confusion_matrix_knn = sklearn.metrics.confusion_matrix(test_labels, pred_knn)
tn_knn, fp_knn, fn_knn, tp_knn = confusion_matrix_knn.ravel()

Precision_knn = tp_knn/(tp_knn+fp_knn)
Recall_knn = tp_knn/(tp_knn+fn_knn)
F1_knn = 2*((Precision_knn*Recall_knn)/(Precision_knn+Recall_knn))

print(f"Precision KNN = {Precision_knn}")
print(f"Recall KNN = {Recall_knn}")
print(f"F1 Score KNN = {F1_knn}")


Precision KNN = 0.4671717171717172
Recall KNN = 0.27788208787082236
F1 Score KNN = 0.34848128090416763


### XGBoost Classifier

In [36]:
 from scipy import stats
 from scipy.stats import randint
 from sklearn.model_selection import RandomizedSearchCV
 from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score

In [37]:
model = XGBClassifier(
 objective= 'binary:logistic')

In [38]:
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4]
             }



In [39]:
clf = RandomizedSearchCV(model, 
                         param_distributions = param_dist,
                         n_iter = 5,
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

In [40]:
clf.fit(train_data, train_labels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:   38.3s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   44.8s finished
[14:58:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(error_score=0,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estima...
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026CFD46F190>,
                                        'lear

In [41]:
pred_xgb = clf.predict(test_data)

In [42]:
errors_xgb = abs(pred_xgb - test_labels)
errors_xgb = [x for x in errors_xgb if x != 0]

In [43]:
print(f"Accuracy XGB = {(len(test_labels)-len(errors_xgb))/len(test_labels)}")

Accuracy XGB = 0.7419817604262732


In [44]:
confusion_matrix_xgb = sklearn.metrics.confusion_matrix(test_labels, pred_xgb)
tn_xgb, fp_xgb, fn_xgb, tp_xgb = confusion_matrix_xgb.ravel()

Precision_xgb = tp_xgb/(tp_xgb+fp_xgb)
Recall_xgb = tp_xgb/(tp_xgb+fn_xgb)
F1_xgb = 2*((Precision_xgb*Recall_xgb)/(Precision_xgb+Recall_xgb))

print(f"Precision xgb = {Precision_xgb}")
print(f"Recall xgb = {Recall_xgb}")
print(f"F1 Score xgb = {F1_xgb}")


Precision xgb = 0.5725
Recall xgb = 0.21498310176492677
F1 Score xgb = 0.3125853125853126


## Funktion für Applikation

In [45]:
# for x in range(1,8):
#      print(model.predict_proba(np.array([[2,0,2,0,1,2,x]])).reshape((1,-1)))
        
        
def vorschlag(kunde):
    kunde = np.array(kunde).reshape((1,-1))
    
    kunde[0][6] = 1
    prob1 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 2
    prob2 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 3
    prob3 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 4
    prob4 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 5
    prob5 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 6
    prob6 = model.predict_proba(kunde)[0][1]
    
    kunde[0][6] = 7
    prob7 = model.predict_proba(kunde)[0][1]
    
    produkt = {"Girokonto":prob1,"Kredit":prob2, "Tagesgeldkonto":prob3, "Depotkonto":prob4, "Altersvorsorge":prob5, "Versicherung":prob6, "Bausparvertrag":prob7}
    
    print(f"Top 3 Produktvorschläge:")
    print(f"1. {sorted(produkt, key=produkt.get, reverse=True)[:3][0]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][0])*100),2)}% Erfolgschance")
    print(f"2. {sorted(produkt, key=produkt.get, reverse=True)[:3][1]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][1])*100),2)}% Erfolgschance")
    print(f"3. {sorted(produkt, key=produkt.get, reverse=True)[:3][2]} mit {round((produkt.get(sorted(produkt, key=produkt.get, reverse=True)[:3][2])*100),2)}% Erfolgschance")
    

In [46]:
#vorschlag(np.array([[2,0,2,0,1,2,x]]).reshape((1,-1)))


In [53]:
filename = 'jungbank_xgb.sav'
pickle.dump(clf, open(filename, 'wb'))

In [48]:
dfTest.head()

,Geschlecht,Job,Familienstand,Kinder,Gekauft,Altersgruppe,balance_cat,Produkt
0,1,1,2,1,0,3,4,4
1,1,7,1,1,0,3,7,4
2,2,4,2,0,0,1,3,2
3,1,5,1,1,0,2,3,5
4,2,4,1,1,0,3,5,5
